# Informações Sobre os Dados

# Instalando Bibliotecas Adicionais

In [1]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip &> /dev/null

# Importando Bibliotecas

In [2]:
import altair as alt
import numpy as np
import pandas as pd
from ipywidgets import widgets
from pandas_profiling import ProfileReport

# Carregando os Dados

Carregando os dados aglutinados de desmatamento e casos de dengue, para pegar a lista de municípios.

In [3]:
df_aglut = pd.read_csv(
    f'https://raw.githubusercontent.com/george-gca/MO826_IA368_CF044/main/data/processed/dados_conjuntos.csv')

cities = set(df_aglut.Municipio.unique())
print(f'{len(cities)} municípios')

740 municípios


Criando dicionário que associa município ao seu respectivo estado

In [4]:
states = set(df_aglut.Estado.unique())
state_dict = df_aglut[['Municipio', 'Estado']].set_index('Municipio').to_dict()['Estado']

In [5]:
years = list(range(2010, 2020))
print(f'Anos a serem considerados: {years}')

Anos a serem considerados: [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [6]:
def load_csv_info(group, years, cities, state_dict):
  dfs = []
  for year in years:
    df = pd.read_csv(
        f'https://raw.githubusercontent.com/george-gca/MO826_IA368_CF044/main/data/external/saneamento/{year}_{group}.csv',
        sep='\t')

    # remove line breaks from columns titles
    columns = [c.replace('\n', ' ') for c in df.columns]
    df.columns = columns

    # remove (Município) (code) from city name, rename Localidade to Municipio
    df.Localidade = df.Localidade.str.split().str[:-2]
    df.Localidade = df.Localidade.str.join(' ')
    df.rename(columns={'Localidade': 'Municipio'}, inplace=True)

    # keep only cities that are in the previous list of cities
    df = df[df.Municipio.isin(cities)]
    
    # fill state information
    # TODO: fix complains about slicing
    df['Estado'] = df.Municipio
    df['Estado'] = df.Estado.replace(state_dict)
    df['Ano'] = year

    # reorder columns to put Municipio, Estado, and Ano at first
    columns = list(df.columns)
    columns = columns[:1] + columns[-2:] + columns[1:-2]
    df = df[columns]

    # remove '%' and '.', replace ',' by '.' for numeric types
    for c in df.columns[3:]:
      if df[c].dtype == object:
        df[c] = df[c].str.replace('%', '')
        df[c] = df[c].str.replace('.', '')
        df[c] = df[c].str.replace(',', '.')

    dfs.append(df)

  return pd.concat(dfs, ignore_index=True)

In [7]:
df_01 = load_csv_info('01', years, cities, state_dict)
df_01

,Municipio,Estado,Ano,População total (pessoas) (IBGE),População das cidades declarantes do SNIS (pessoas) (SNIS),População urbana total (pessoas) (SNIS),Área do município (km²) (IBGE),População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS),População urbana que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS),Parcela da população urbana que mora em domicílios sem acesso ao serviço de coleta de esgoto (% da população) (SNIS),Parcela da população total que mora em domicílios sem acesso ao serviço de coleta de esgoto (% da população) (SNIS)
0,Ariquemes,RO,2010,90353,90353,76525,4426.570,90353,76525,100.0,100.0
1,Cacoal,RO,2010,78574,78574,61921,3792.950,42574,25921,41.9,54.2
2,Jaru,RO,2010,52005,52005,35118,2944.130,52005,35118,100.0,100.0
3,Ji-Paraná,RO,2010,116610,116610,104858,6896.650,116610,104858,100.0,100.0
4,Porto Velho,RO,2010,428527,428527,390733,34090.960,422016,384222,98.3,98.5
...,...,...,...,...,...,...,...,...,...,...,...
1265,Santo Antônio do Leverger,MT,2019,16628,-,6448,11735.750,-,-,-,-
1266,Sinop,MT,2019,142996,142996,118536,3942.230,100677,76217,64.3,70.4
1267,Sorriso,MT,2019,90313,90313,79239,9347.560,60306,49232,62.1,66.8
1268,Tangará da Serra,MT,2019,103750,103750,94411,11601.250,74224,64885,68.7,71.5


In [8]:
df_02 = load_csv_info('02', years, cities, state_dict)
df_02

,Municipio,Estado,Ano,Internações por doenças gastrointestinais infecciosas (Número de internações) (DATASUS),Internações por febre amarela (Número de internações) (DATASUS),Internações por dengue (Número de internações) (DATASUS),Internações por leptospirose (Número de internações) (DATASUS),Internações por malária (Número de internações) (DATASUS),Internações por esquistossomose (Número de internações) (DATASUS),Taxa de incidência de internações por doenças associadas à falta de saneamento (Internações por 10 mil habitantes) (DATASUS),Internações por doenças associadas à falta de saneamento (Número de internações) (DATASUS)
0,Ariquemes,RO,2010,111.0,0,113.0,0,40,0,29.22,264.000
1,Cacoal,RO,2010,222.0,0,179.0,0,6,0,51.80,407.000
2,Jaru,RO,2010,64.0,0,109.0,0,31,0,39.23,204.000
3,Ji-Paraná,RO,2010,461.0,0,524.0,1,39,0,87.90,1.025
4,Porto Velho,RO,2010,510.0,0,255.0,2,259,0,23.94,1.026
...,...,...,...,...,...,...,...,...,...,...,...
1265,Santo Antônio do Leverger,MT,2019,2.0,0,0.0,0,0,0,1.20,2.000
1266,Sinop,MT,2019,21.0,0,9.0,0,3,0,2.31,33.000
1267,Sorriso,MT,2019,26.0,0,2.0,0,3,0,3.43,31.000
1268,Tangará da Serra,MT,2019,133.0,0,17.0,0,1,0,14.55,151.000


In [9]:
df_03 = load_csv_info('03', years, cities, state_dict)
df_03

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Municipio,Estado,Ano,Taxa de incidência de internações de doenças gastrointestinais infecciosas (Internações por 10 mil habitantes) (DATASUS),Taxa de incidência de internações de febre amarela (Internações por 10 mil habitantes) (DATASUS),Taxa de incidência de internações de dengue (Internações por 10 mil habitantes) (DATASUS),Taxa de incidência de internações de leptospirose (Internações por 10 mil habitantes) (DATASUS),Taxa de incidência de internações de malária (Internações por 10 mil habitantes) (DATASUS),Taxa de incidência de internações de esquistossomose (Internações por 10 mil habitantes) (DATASUS),Internações por doenças respiratórias: gripes e pneumonias (Número de internações) (DATASUS),Incidência de internações por doenças respiratórias: gripes e pneumonias (Internações por 10 mil habitantes) (DATASUS)
0,Ariquemes,RO,2010,12.29,0.00,12.51,0.00,4.43,0.00,397.0,43.94
1,Cacoal,RO,2010,28.25,0.00,22.78,0.00,0.76,0.00,450.0,57.27
2,Jaru,RO,2010,12.31,0.00,20.96,0.00,5.96,0.00,162.0,31.15
3,Ji-Paraná,RO,2010,39.53,0.00,44.94,0.09,3.34,0.00,766.0,65.69
4,Porto Velho,RO,2010,11.90,0.00,5.95,0.05,6.04,0.00,979.0,22.85
...,...,...,...,...,...,...,...,...,...,...,...
1265,Santo Antônio do Leverger,MT,2019,1.20,0.00,0.00,0.00,0.00,0.00,21.0,12.63
1266,Sinop,MT,2019,1.47,0.00,0.63,0.00,0.21,0.00,102.0,7.13
1267,Sorriso,MT,2019,2.88,0.00,0.22,0.00,0.33,0.00,147.0,16.28
1268,Tangará da Serra,MT,2019,12.82,0.00,1.64,0.00,0.10,0.00,436.0,42.02


In [10]:
df_04 = load_csv_info('04', years, cities, state_dict)
df_04

,Municipio,Estado,Ano,"Extensão da rede de distribuição de água, em km (km) (SNIS)","Extensão da rede de distribuição de esgoto, em km (km) (SNIS)","Investimentos realizados pelo prestador de serviços, em R$ de 2017 (R$ a preços de 2019) (ITB)","Investimentos realizados pelo município, em R$ de 2017 (R$ a preços de 2019) (ITB)","Investimentos realizados pelo governo estadual, em R$ de 2017 (R$ a preços de 2019) (ITB)",Investimentos totais em saneamento (R$ a preços de 2019) (ITB),Investimentos per capita em saneamento (R$ a preços de 2019) (ITB)
0,Ariquemes,RO,2010,182.20,0.00,311124.09,0.00,0.00,311124.09,3.44
1,Cacoal,RO,2010,407.00,158.00,964646.18,0.00,0.00,964646.18,12.28
2,Jaru,RO,2010,91.10,0.00,285941.82,0.00,0.00,285941.82,5.50
3,Ji-Paraná,RO,2010,282.70,0.00,509546.33,0.00,0.00,509546.33,4.37
4,Porto Velho,RO,2010,881.50,30.00,1825895.75,0.00,0.00,1825895.75,4.26
...,...,...,...,...,...,...,...,...,...,...
1265,Santo Antônio do Leverger,MT,2019,-,-,0.00,0.00,0.00,0.00,0.00
1266,Sinop,MT,2019,952.56,225.61,7640351.90,0.00,0.00,7640351.90,53.43
1267,Sorriso,MT,2019,462.89,311.28,10720087.36,0.00,0.00,10720087.36,118.70
1268,Tangará da Serra,MT,2019,355.15,138.34,1239501.58,0.00,0.00,1239501.58,11.95


In [11]:
df_05 = load_csv_info('05', years, cities, state_dict)
df_05

,Municipio,Estado,Ano,Número de moradias permanentes (habitações) (IBGE),Número de moradias permanentes em áreas urbanas (habitações) (IBGE),Número de pessoas por moradia (pessoas por moradia) (IBGE),Número de pessoas por moradia (área urbana) (pessoas por moradia) (IBGE),"PIB, em R$ milhões a preços de 2017 (R$ milhões a preços de 2019) (IBGE)","PIB per capita, em R$ a preços de 2017 (R$ por habitante a preços de 2019) (IBGE)",Área do município (km²) (IBGE),Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE)
0,Ariquemes,RO,2010,27241,22976,3.32,3.33,2229.460,24674.98,4426.570,20.41
1,Cacoal,RO,2010,24249,19416,3.24,3.19,1938.340,24668.96,3792.950,20.72
2,Jaru,RO,2010,15946,10967,3.26,3.20,1302.520,25046.01,2944.130,17.66
3,Ji-Paraná,RO,2010,35652,32278,3.27,3.25,3031.040,25992.99,6896.650,16.91
4,Porto Velho,RO,2010,117142,108175,3.66,3.61,14856.300,34668.30,34090.960,12.57
...,...,...,...,...,...,...,...,...,...,...,...
1265,Santo Antônio do Leverger,MT,2019,8441,3070,1.97,2.10,-,-,11735.750,1.42
1266,Sinop,MT,2019,43864,37630,3.26,3.15,-,-,3942.230,36.27
1267,Sorriso,MT,2019,25804,23169,3.50,3.42,-,-,9347.560,9.66
1268,Tangará da Serra,MT,2019,32937,30954,3.15,3.05,-,-,11601.250,8.94


# Convertendo os Tipos de Dados

In [12]:
def convert_dtypes(df):
  df.replace({'-': np.NaN}, inplace=True)
  for c in df.columns[3:]:
    # if c.startswith('Área') or c.startswith('Parcela'):
    if df[c].dtype == object:
      df[c] = df[c].astype(float)
    # else:
      # df[c] = df[c].astype(int)
  return df

In [13]:
df_01 = convert_dtypes(df_01)
df_02 = convert_dtypes(df_02)
df_03 = convert_dtypes(df_03)
df_04 = convert_dtypes(df_04)
df_05 = convert_dtypes(df_05)

In [14]:
print('Dados 01')
for state in states:
  df_tmp = df_01[df_01.Estado == state.upper()]
  print(f'Total de Municípios em {state.upper()}: {len(df_tmp)}')
  print(f'Total de Municípios Completos em {state.upper()}: {len(df_tmp.dropna())}\n')

Dados 01
Total de Municípios em AM: 150
Total de Municípios Completos em AM: 82

Total de Municípios em MA: 230
Total de Municípios Completos em MA: 196

Total de Municípios em AP: 20
Total de Municípios Completos em AP: 20

Total de Municípios em TO: 90
Total de Municípios Completos em TO: 90

Total de Municípios em RR: 20
Total de Municípios Completos em RR: 20

Total de Municípios em AC: 20
Total de Municípios Completos em AC: 19

Total de Municípios em PA: 460
Total de Municípios Completos em PA: 379

Total de Municípios em MT: 200
Total de Municípios Completos em MT: 150

Total de Municípios em RO: 80
Total de Municípios Completos em RO: 76



In [15]:
print('Dados 02')
for state in states:
  df_tmp = df_02[df_02.Estado == state.upper()]
  print(f'Total de Municípios em {state.upper()}: {len(df_tmp)}')
  print(f'Total de Municípios Completos em {state.upper()}: {len(df_tmp.dropna())}\n')

Dados 02
Total de Municípios em AM: 150
Total de Municípios Completos em AM: 150

Total de Municípios em MA: 230
Total de Municípios Completos em MA: 230

Total de Municípios em AP: 20
Total de Municípios Completos em AP: 20

Total de Municípios em TO: 90
Total de Municípios Completos em TO: 90

Total de Municípios em RR: 20
Total de Municípios Completos em RR: 20

Total de Municípios em AC: 20
Total de Municípios Completos em AC: 20

Total de Municípios em PA: 460
Total de Municípios Completos em PA: 460

Total de Municípios em MT: 200
Total de Municípios Completos em MT: 200

Total de Municípios em RO: 80
Total de Municípios Completos em RO: 80



In [16]:
print('Dados 03')
for state in states:
  df_tmp = df_03[df_03.Estado == state.upper()]
  print(f'Total de Municípios em {state.upper()}: {len(df_tmp)}')
  print(f'Total de Municípios Completos em {state.upper()}: {len(df_tmp.dropna())}\n')

Dados 03
Total de Municípios em AM: 150
Total de Municípios Completos em AM: 150

Total de Municípios em MA: 230
Total de Municípios Completos em MA: 230

Total de Municípios em AP: 20
Total de Municípios Completos em AP: 20

Total de Municípios em TO: 90
Total de Municípios Completos em TO: 90

Total de Municípios em RR: 20
Total de Municípios Completos em RR: 20

Total de Municípios em AC: 20
Total de Municípios Completos em AC: 20

Total de Municípios em PA: 460
Total de Municípios Completos em PA: 460

Total de Municípios em MT: 200
Total de Municípios Completos em MT: 200

Total de Municípios em RO: 80
Total de Municípios Completos em RO: 80



In [17]:
print('Dados 04')
for state in states:
  df_tmp = df_04[df_04.Estado == state.upper()]
  print(f'Total de Municípios em {state.upper()}: {len(df_tmp)}')
  print(f'Total de Municípios Completos em {state.upper()}: {len(df_tmp.dropna())}\n')

Dados 04
Total de Municípios em AM: 150
Total de Municípios Completos em AM: 78

Total de Municípios em MA: 230
Total de Municípios Completos em MA: 183

Total de Municípios em AP: 20
Total de Municípios Completos em AP: 20

Total de Municípios em TO: 90
Total de Municípios Completos em TO: 90

Total de Municípios em RR: 20
Total de Municípios Completos em RR: 20

Total de Municípios em AC: 20
Total de Municípios Completos em AC: 18

Total de Municípios em PA: 460
Total de Municípios Completos em PA: 350

Total de Municípios em MT: 200
Total de Municípios Completos em MT: 144

Total de Municípios em RO: 80
Total de Municípios Completos em RO: 71



In [18]:
print('Dados 05')
for state in states:
  df_tmp = df_05[df_05.Estado == state.upper()]
  print(f'Total de Municípios em {state.upper()}: {len(df_tmp)}')
  print(f'Total de Municípios Completos em {state.upper()}: {len(df_tmp.dropna())}\n')

Dados 05
Total de Municípios em AM: 150
Total de Municípios Completos em AM: 135

Total de Municípios em MA: 230
Total de Municípios Completos em MA: 207

Total de Municípios em AP: 20
Total de Municípios Completos em AP: 18

Total de Municípios em TO: 90
Total de Municípios Completos em TO: 81

Total de Municípios em RR: 20
Total de Municípios Completos em RR: 18

Total de Municípios em AC: 20
Total de Municípios Completos em AC: 18

Total de Municípios em PA: 460
Total de Municípios Completos em PA: 414

Total de Municípios em MT: 200
Total de Municípios Completos em MT: 180

Total de Municípios em RO: 80
Total de Municípios Completos em RO: 72



# Gerando Estatísticas Básicas Sobre os Dados

Quantos municípios possuem cada estado na base de dados

In [19]:
states = df_01.Estado.unique()
for state in states:
  print(f'{state} possui {len(df_01[df_01.Estado == state].Municipio.unique())} municípios no grupo 01')
  print(f'{state} possui {len(df_02[df_02.Estado == state].Municipio.unique())} municípios no grupo 02')
  print(f'{state} possui {len(df_03[df_03.Estado == state].Municipio.unique())} municípios no grupo 03')
  print(f'{state} possui {len(df_04[df_04.Estado == state].Municipio.unique())} municípios no grupo 04')
  print(f'{state} possui {len(df_05[df_05.Estado == state].Municipio.unique())} municípios no grupo 05\n')

RO possui 8 municípios no grupo 01
RO possui 8 municípios no grupo 02
RO possui 8 municípios no grupo 03
RO possui 8 municípios no grupo 04
RO possui 8 municípios no grupo 05

AC possui 2 municípios no grupo 01
AC possui 2 municípios no grupo 02
AC possui 2 municípios no grupo 03
AC possui 2 municípios no grupo 04
AC possui 2 municípios no grupo 05

AM possui 15 municípios no grupo 01
AM possui 15 municípios no grupo 02
AM possui 15 municípios no grupo 03
AM possui 15 municípios no grupo 04
AM possui 15 municípios no grupo 05

RR possui 2 municípios no grupo 01
RR possui 2 municípios no grupo 02
RR possui 2 municípios no grupo 03
RR possui 2 municípios no grupo 04
RR possui 2 municípios no grupo 05

PA possui 45 municípios no grupo 01
PA possui 45 municípios no grupo 02
PA possui 45 municípios no grupo 03
PA possui 45 municípios no grupo 04
PA possui 45 municípios no grupo 05

AP possui 2 municípios no grupo 01
AP possui 2 municípios no grupo 02
AP possui 2 municípios no grupo 03
AP po

# Filtrando Colunas de Interesse e Aglutinando os Dados

In [20]:
df_01_columns = list(df_01.columns)
df_01_columns = df_01_columns[:3] + \
  ['População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS)',
   'Parcela da população total que mora em domicílios sem acesso ao serviço de coleta de esgoto (% da população) (SNIS)']

df_04_columns = ['Extensão da rede de distribuição de esgoto, em km (km) (SNIS)']

df_05_columns = ['Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE)']

df_useful = pd.concat([df_01[df_01_columns], df_04[df_04_columns], df_05[df_05_columns]], axis=1)
df_useful

,Municipio,Estado,Ano,População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS),Parcela da população total que mora em domicílios sem acesso ao serviço de coleta de esgoto (% da população) (SNIS),"Extensão da rede de distribuição de esgoto, em km (km) (SNIS)",Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE)
0,Ariquemes,RO,2010,90353.0,100.0,0.00,20.41
1,Cacoal,RO,2010,42574.0,54.2,158.00,20.72
2,Jaru,RO,2010,52005.0,100.0,0.00,17.66
3,Ji-Paraná,RO,2010,116610.0,100.0,0.00,16.91
4,Porto Velho,RO,2010,422016.0,98.5,30.00,12.57
...,...,...,...,...,...,...,...
1265,Santo Antônio do Leverger,MT,2019,NaN,NaN,NaN,1.42
1266,Sinop,MT,2019,100677.0,70.4,225.61,36.27
1267,Sorriso,MT,2019,60306.0,66.8,311.28,9.66
1268,Tangará da Serra,MT,2019,74224.0,71.5,138.34,8.94


In [21]:
df_useful.Municipio.unique()

array(['Ariquemes', 'Cacoal', 'Jaru', 'Ji-Paraná', 'Porto Velho',
       'Rolim de Moura', 'Vilhena', 'Cruzeiro do Sul', 'Rio Branco',
       'Careiro da Várzea', 'Coari', 'Humaitá', 'Iranduba', 'Itacoatiara',
       'Manacapuru', 'Manaus', 'Manicoré', 'Maués', 'Novo Airão',
       'Parintins', 'Presidente Figueiredo', 'Rio Preto da Eva',
       'Tabatinga', 'Tefé', 'Boa Vista', 'Abaetetuba', 'Acará',
       'Alenquer', 'Altamira', 'Ananindeua', 'Barcarena', 'Belém',
       'Benevides', 'Bragança', 'Breu Branco', 'Breves', 'Cametá',
       'Capanema', 'Capitão Poço', 'Castanhal', 'Dom Eliseu',
       'Igarapé-Miri', 'Ipixuna do Pará', 'Itaituba', 'Itupiranga',
       'Jacundá', 'Juruti', 'Marabá', 'Marituba', 'Moju', 'Monte Alegre',
       'Novo Repartimento', 'Óbidos', 'Oriximiná', 'Paragominas',
       'Parauapebas', 'Portel', 'Redenção', 'Rondon do Pará',
       'Santa Bárbara do Pará', 'Santana do Araguaia', 'Santarém',
       'São Félix do Xingu', 'São Miguel do Guamá', 'Tailândia

In [22]:
df_aglut = pd.read_csv(
    f'https://raw.githubusercontent.com/george-gca/MO826_IA368_CF044/main/data/processed/dados_conjuntos_col_ano.csv')

df_aglut

,Lat,Long,Municipio,Estado,AreaKm2,Ano,Desmatado,Incremento,Floresta,NaoFloresta,Hidrografia,PercDesmatado,Internações - Dengue,Internações Ano Seguinte - Dengue
0,-1.73866,-48.85524,Abaetetuba,PA,1617,2008,683.3,1.2,505.6,13.6,413.6,0.422573,88.0,52.0
1,-4.96265,-48.39406,Abel Figueiredo,PA,616,2008,561.5,2.0,54.3,0.0,0.0,0.911526,12.0,25.0
2,-9.46606,-49.33901,Abreulândia,TO,1900,2008,7.4,1.9,47.9,1844.3,0.0,0.003895,0.0,0.0
3,-2.03379,-48.31904,Acará,PA,4361,2008,2263.7,47.9,1971.3,43.3,70.5,0.519078,86.0,57.0
4,-15.19422,-56.31716,Acorizal,MT,841,2008,0.0,0.0,0.0,840.8,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8875,-6.84864,-49.25551,Xinguara,PA,3793,2019,3381.1,1.2,394.3,0.0,17.6,0.891405,17.0,NaN
8876,-3.25303,-45.97435,Zé Doca,MA,2438,2019,2021.6,1.1,376.2,0.0,0.0,0.829204,8.0,NaN
8877,-6.76170,-50.32800,Água Azul do Norte,PA,7125,2019,4674.0,9.7,2323.1,127.9,0.0,0.656000,35.0,NaN
8878,-14.08771,-52.43685,Água Boa,MT,7486,2019,40.1,0.0,123.5,7322.4,0.0,0.005357,10.0,NaN


In [23]:
cities_with_na = set(df_aglut[df_aglut['Internações - Dengue'].isna()].Municipio.unique())
print(f'Municípios com dados de dengue faltantes: {len(cities_with_na)}')
cities_with_na

Municípios com dados de dengue faltantes: 33


{'Alto Alegre',
 'Amajari',
 'Amapá',
 'Boa Vista',
 'Bonfim',
 'Calçoene',
 'Cantá',
 'Caracaraí',
 'Caroebe',
 'Curuá',
 'Ferreira Gomes',
 'Guiratinga',
 'Iracema',
 'Itaubal',
 'Laranjal do Jari',
 'Macapá',
 'Mazagão',
 'Mucajaí',
 'Normandia',
 'Oiapoque',
 'Pacaraima',
 'Pedra Branca do Amapari',
 'Porto Grande',
 'Rorainópolis',
 'Santana',
 'Serra do Navio',
 'São João da Baliza',
 'São Luiz',
 'Taipas do Tocantins',
 'Tartarugalzinho',
 'Uiramutã',
 'Urucurituba',
 'Vitória do Jari'}

In [24]:
df_aglut_clean = df_aglut[~df_aglut.Municipio.isin(cities_with_na)]
df_aglut_clean

,Lat,Long,Municipio,Estado,AreaKm2,Ano,Desmatado,Incremento,Floresta,NaoFloresta,Hidrografia,PercDesmatado,Internações - Dengue,Internações Ano Seguinte - Dengue
0,-1.73866,-48.85524,Abaetetuba,PA,1617,2008,683.3,1.2,505.6,13.6,413.6,0.422573,88.0,52.0
1,-4.96265,-48.39406,Abel Figueiredo,PA,616,2008,561.5,2.0,54.3,0.0,0.0,0.911526,12.0,25.0
2,-9.46606,-49.33901,Abreulândia,TO,1900,2008,7.4,1.9,47.9,1844.3,0.0,0.003895,0.0,0.0
3,-2.03379,-48.31904,Acará,PA,4361,2008,2263.7,47.9,1971.3,43.3,70.5,0.519078,86.0,57.0
4,-15.19422,-56.31716,Acorizal,MT,841,2008,0.0,0.0,0.0,840.8,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8875,-6.84864,-49.25551,Xinguara,PA,3793,2019,3381.1,1.2,394.3,0.0,17.6,0.891405,17.0,NaN
8876,-3.25303,-45.97435,Zé Doca,MA,2438,2019,2021.6,1.1,376.2,0.0,0.0,0.829204,8.0,NaN
8877,-6.76170,-50.32800,Água Azul do Norte,PA,7125,2019,4674.0,9.7,2323.1,127.9,0.0,0.656000,35.0,NaN
8878,-14.08771,-52.43685,Água Boa,MT,7486,2019,40.1,0.0,123.5,7322.4,0.0,0.005357,10.0,NaN


In [25]:
df_merged = pd.merge(df_useful, df_aglut_clean, how='inner', on=['Municipio', 'Estado', 'Ano'])
df_merged

,Municipio,Estado,Ano,População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS),Parcela da população total que mora em domicílios sem acesso ao serviço de coleta de esgoto (% da população) (SNIS),"Extensão da rede de distribuição de esgoto, em km (km) (SNIS)",Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE),Lat,Long,AreaKm2,Desmatado,Incremento,Floresta,NaoFloresta,Hidrografia,PercDesmatado,Internações - Dengue,Internações Ano Seguinte - Dengue
0,Ariquemes,RO,2010,90353.0,100.0,0.00,20.41,-9.97866,-63.05645,4481,3175.0,4.3,1216.2,89.6,0.2,0.708547,107.0,25.0
1,Cacoal,RO,2010,42574.0,54.2,158.00,20.72,-11.30292,-61.25045,3821,2490.9,1.7,1295.4,26.2,5.8,0.651897,164.0,20.0
2,Jaru,RO,2010,52005.0,100.0,0.00,17.66,-10.64842,-62.55320,2976,2616.3,0.2,314.8,44.8,0.0,0.879133,108.0,43.0
3,Ji-Paraná,RO,2010,116610.0,100.0,0.00,16.91,-10.44844,-61.69061,6955,2601.2,0.8,4250.0,66.3,35.3,0.374004,514.0,192.0
4,Porto Velho,RO,2010,422016.0,98.5,30.00,12.57,-8.98448,-63.99115,34631,7797.0,135.0,25574.8,431.9,751.1,0.225145,228.0,78.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1225,Santo Antônio do Leverger,MT,2019,NaN,NaN,NaN,1.42,-16.48613,-55.26485,12262,1047.7,1.4,1093.1,10099.5,21.7,0.085443,0.0,NaN
1226,Sinop,MT,2019,100677.0,70.4,225.61,36.27,-11.74201,-55.40265,3935,2609.9,72.2,1170.4,136.3,18.4,0.663253,6.0,NaN
1227,Sorriso,MT,2019,60306.0,66.8,311.28,9.66,-12.69331,-55.77741,9349,3977.4,0.2,1252.5,4119.1,0.0,0.425436,2.0,NaN
1228,Tangará da Serra,MT,2019,74224.0,71.5,138.34,8.94,-14.43270,-58.38021,11333,2184.1,3.7,635.0,8512.8,0.9,0.192720,17.0,NaN


In [26]:
# remove todo mundo que já foi 100% desmatado em 2008 e não tem mais área pra 
# desmatar antes do último ano da análise
cities_filtered = set(df_merged.loc[
                      (df_merged.PercDesmatado < 0.6) &
                      (df_merged.Ano == 2010),:].Municipio.unique())
cities_filtered = set(df_merged.loc[
                      (df_merged.PercDesmatado < 1) &
                      (df_merged.Ano == 2018) &
                      (df_merged.Municipio.isin(cities_filtered)),:].Municipio.unique())

print(f'Municípios antes do filtro: {len(df_merged.Municipio.unique())}')
print(f'Municípios após o filtro: {len(cities_filtered)}')
# cities_filtered

Municípios antes do filtro: 119
Municípios após o filtro: 89


In [27]:
df_merged_filtered = df_merged[df_merged.Municipio.isin(cities_filtered)]
df_merged_filtered

,Municipio,Estado,Ano,População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS),Parcela da população total que mora em domicílios sem acesso ao serviço de coleta de esgoto (% da população) (SNIS),"Extensão da rede de distribuição de esgoto, em km (km) (SNIS)",Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE),Lat,Long,AreaKm2,Desmatado,Incremento,Floresta,NaoFloresta,Hidrografia,PercDesmatado,Internações - Dengue,Internações Ano Seguinte - Dengue
3,Ji-Paraná,RO,2010,116610.0,100.0,0.00,16.91,-10.44844,-61.69061,6955,2601.2,0.8,4250.0,66.3,35.3,0.374004,514.0,192.0
4,Porto Velho,RO,2010,422016.0,98.5,30.00,12.57,-8.98448,-63.99115,34631,7797.0,135.0,25574.8,431.9,751.1,0.225145,228.0,78.0
6,Vilhena,RO,2010,76202.0,100.0,0.00,6.51,-12.12351,-60.11211,11584,1730.2,8.8,8162.7,1685.8,1.5,0.149361,112.0,46.0
7,Cruzeiro do Sul,AC,2010,78507.0,100.0,0.00,8.94,-7.95559,-72.73617,9202,982.2,23.0,7973.3,20.7,0.0,0.106738,10.0,13.0
8,Rio Branco,AC,2010,268038.0,79.8,231.00,38.04,-9.99527,-68.15925,9113,2522.7,21.0,6584.8,0.0,0.1,0.276824,367.0,193.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,Rosário Oeste,MT,2019,NaN,NaN,NaN,2.31,-14.86180,-56.25158,7493,66.0,0.2,23.0,7395.0,9.0,0.008808,19.0,NaN
1225,Santo Antônio do Leverger,MT,2019,NaN,NaN,NaN,1.42,-16.48613,-55.26485,12262,1047.7,1.4,1093.1,10099.5,21.7,0.085443,0.0,NaN
1227,Sorriso,MT,2019,60306.0,66.8,311.28,9.66,-12.69331,-55.77741,9349,3977.4,0.2,1252.5,4119.1,0.0,0.425436,2.0,NaN
1228,Tangará da Serra,MT,2019,74224.0,71.5,138.34,8.94,-14.43270,-58.38021,11333,2184.1,3.7,635.0,8512.8,0.9,0.192720,17.0,NaN


Salvandos os dados por município

In [28]:
df_merged.to_csv('desmat_dengue_san.csv', index=False)
df_merged_filtered.to_csv('desmat_dengue_san_filtrado.csv', index=False)

Gerando os dados agrupados por estado

In [29]:
df_merged_states = df_merged.groupby(['Estado', 'Ano']).sum().reset_index()
columns_to_remove = {'Municipio', 'Lat', 'Long',
                     'Parcela da população total que mora em domicílios sem acesso ao serviço de coleta de esgoto (% da população) (SNIS)'}
columns = [c for c in df_merged_states.columns if c not in columns_to_remove]
print(f'Deixando somente as colunas:\n{columns}\n')
df_merged_states = df_merged_states[columns]
df_merged_states

Deixando somente as colunas:
['Estado', 'Ano', 'População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS)', 'Extensão da rede de distribuição de esgoto, em km (km) (SNIS)', 'Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE)', 'AreaKm2', 'Desmatado', 'Incremento', 'Floresta', 'NaoFloresta', 'Hidrografia', 'PercDesmatado', 'Internações - Dengue', 'Internações Ano Seguinte - Dengue']



,Estado,Ano,População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS),"Extensão da rede de distribuição de esgoto, em km (km) (SNIS)",Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE),AreaKm2,Desmatado,Incremento,Floresta,NaoFloresta,Hidrografia,PercDesmatado,Internações - Dengue,Internações Ano Seguinte - Dengue
0,AC,2010,346545.0,231.00,46.98,18315,3504.9,44.0,14558.1,20.7,0.1,0.383562,377.0,206.0
1,AC,2011,353473.0,231.00,47.76,18315,3561.5,56.6,14376.1,20.7,0.1,0.389759,206.0,53.0
2,AC,2012,345283.0,324.88,48.52,18315,3600.3,38.8,14659.5,20.7,0.1,0.394006,53.0,41.0
3,AC,2013,365525.0,438.97,49.59,18315,3625.8,25.5,14106.2,20.7,0.1,0.396799,41.0,560.0
4,AC,2014,367610.0,467.27,50.41,18315,3671.9,46.1,14343.6,20.7,0.1,0.401846,560.0,268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TO,2015,663975.0,1934.73,6916.78,17956,2805.2,2.6,439.2,14603.6,111.0,0.982895,91.0,72.0
66,TO,2016,656048.0,2233.07,6957.17,17956,2807.3,2.1,437.1,14603.6,111.0,0.984190,72.0,85.0
67,TO,2017,597752.0,2368.21,6995.43,17956,2808.2,0.9,436.2,14603.6,111.0,0.984414,85.0,79.0
68,TO,2018,582076.0,2466.85,7093.43,17956,2808.4,0.2,436.0,14603.6,111.0,0.984464,79.0,330.0


In [30]:
df_merged_states_filtered = df_merged_filtered.groupby(['Estado', 'Ano']).sum().reset_index()
columns_to_remove = {'Municipio', 'Lat', 'Long',
                     'Parcela da população total que mora em domicílios sem acesso ao serviço de coleta de esgoto (% da população) (SNIS)'}
columns = [c for c in df_merged_states_filtered.columns if c not in columns_to_remove]
print(f'Deixando somente as colunas:\n{columns}\n')
df_merged_states_filtered = df_merged_states_filtered[columns]
df_merged_states_filtered

Deixando somente as colunas:
['Estado', 'Ano', 'População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS)', 'Extensão da rede de distribuição de esgoto, em km (km) (SNIS)', 'Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE)', 'AreaKm2', 'Desmatado', 'Incremento', 'Floresta', 'NaoFloresta', 'Hidrografia', 'PercDesmatado', 'Internações - Dengue', 'Internações Ano Seguinte - Dengue']



,Estado,Ano,População total que mora em domicílios sem acesso ao serviço de coleta de esgoto (pessoas) (SNIS),"Extensão da rede de distribuição de esgoto, em km (km) (SNIS)",Densidade demográfica (pessoas por km²) (Pessoas por km²) (IBGE),AreaKm2,Desmatado,Incremento,Floresta,NaoFloresta,Hidrografia,PercDesmatado,Internações - Dengue,Internações Ano Seguinte - Dengue
0,AC,2010,346545.0,231.00,46.98,18315,3504.9,44.0,14558.1,20.7,0.1,0.383562,377.0,206.0
1,AC,2011,353473.0,231.00,47.76,18315,3561.5,56.6,14376.1,20.7,0.1,0.389759,206.0,53.0
2,AC,2012,345283.0,324.88,48.52,18315,3600.3,38.8,14659.5,20.7,0.1,0.394006,53.0,41.0
3,AC,2013,365525.0,438.97,49.59,18315,3625.8,25.5,14106.2,20.7,0.1,0.396799,41.0,560.0
4,AC,2014,367610.0,467.27,50.41,18315,3671.9,46.1,14343.6,20.7,0.1,0.401846,560.0,268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,TO,2015,523611.0,1751.66,6874.24,13937,110.6,0.5,36.3,13706.0,87.1,0.312429,79.0,56.0
66,TO,2016,513125.0,1997.07,6913.90,13937,110.9,0.3,36.0,13706.0,87.1,0.313277,56.0,50.0
67,TO,2017,466153.0,2060.21,6951.44,13937,110.9,0.0,36.0,13706.0,87.1,0.313277,50.0,73.0
68,TO,2018,457203.0,2136.85,7049.06,13937,110.9,0.0,36.0,13706.0,87.1,0.313277,73.0,249.0


Salvando os dados agrupados por estado

In [31]:
df_merged_states.to_csv('desmat_dengue_san_estado.csv', index=False)
df_merged_states_filtered.to_csv('desmat_dengue_san_estado_filtrado.csv', index=False)